In [0]:
import pandas as pd
import math

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Alertas_Negativas_Twitter/notebooks")

In [25]:
df_user_info_final=pd.read_pickle("../data/raw/df_user_info_final.pkl")
df_tweets_total=pd.read_pickle("../data/raw/df_tweets_total.pkl")
df_tweets_total=df_tweets_total[df_tweets_total["screen_name"]!="bcrpoficial"].copy()
df_tweets_total.shape

(28859, 21)

In [26]:
df_tweets_counts=df_tweets_total.groupby(by="screen_name").text.count().to_frame().sort_values("text",ascending=False)
df_tweets_counts.rename(columns={"text":"tweets_counts"},inplace=True)

df_retweets_counts=df_tweets_total.groupby(by="screen_name").retweets.apply(lambda x : sum(x)).to_frame().sort_values("retweets",ascending=False).reset_index()
df_retweets_counts.rename(columns={"retweets":"retweets_counts"},inplace=True)

df_likes_counts=df_tweets_total.groupby(by="screen_name").likes.apply(lambda x : sum(x)).to_frame().sort_values("likes",ascending=False).reset_index()
df_likes_counts.rename(columns={"likes":"likes_counts"},inplace=True)

df_replies_counts=df_tweets_total.groupby(by="screen_name").replies.apply(lambda x : sum(x)).to_frame().sort_values("replies",ascending=False).reset_index()
df_replies_counts.rename(columns={"replies":"replies_counts"},inplace=True)

aux=pd.merge(pd.merge(pd.merge(df_tweets_counts,df_retweets_counts,how="inner",on="screen_name")\
                      ,df_likes_counts,how="inner",on="screen_name")\
                      ,df_replies_counts,how="inner", on="screen_name")
print(aux.shape)
aux.head()

(10936, 5)


,screen_name,tweets_counts,retweets_counts,likes_counts,replies_counts
0,Agencia_Andina,569,1180,2401,460
1,Gestionpe,542,2070,2342,401
2,BizRepublicNews,273,18,23,3
3,ExpresoPeru,209,238,235,55
4,proactivoperu,155,20,36,0


In [27]:
df_user_info_influencers=pd.merge(df_user_info_final,aux,how="inner",on="screen_name")
print(df_user_info_influencers.shape)
df_user_info_influencers.head()

(10850, 16)


,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes,is_followers,is_following,tweets_counts,retweets_counts,likes_counts,replies_counts
0,FelizDiaLima,#FelizDíaLima. 👑 👑 👑 ⭐️,,,2020-01-16,1218063509549993986,91,99,5,165,0,0,1,0,0,0
1,JuanCar99215124,Juan Carlos Miranda,,,2020-01-13,1216755584080347137,22,34,3,137,0,0,1,0,0,0
2,cocorocovillano,coqui villano diarrea,Jesus maria,,2020-01-10,1215664651813388289,200,249,49,342,0,0,2,0,3,2
3,CarasasJonathan,Jonathan Carasas C.,"Montevideo, Uruguay",elojocriticojcc.blogspot.com,2019-12-27,1210664626666520581,406,78,8,228,0,0,1,0,0,0
4,pedropablofb,Pedro Pablo,,,2019-12-26,1210126217539706886,5,99,4,2,0,0,1,0,0,1


In [0]:
features=["followers","tweets_counts","likes_counts","retweets_counts","replies_counts"]
gains=[]
for ft in features:
  sum_total=df_user_info_influencers[ft].sum()
  gain=-sum([(i/sum_total)*math.log2(i/sum_total) for i in df_user_info_influencers[ft] if i!=0])
  gains.append(gain)
df_user_info_influencers["metrics"]=df_user_info_influencers["followers"]*gains[0]+df_user_info_influencers["tweets_counts"]*gains[1]+df_user_info_influencers["likes_counts"]*gains[2]+df_user_info_influencers["retweets_counts"]*gains[3]+df_user_info_influencers["replies_counts"]*gains[4]

In [29]:
gains

[7.417320332141926,
 12.086904128210957,
 7.302639999700798,
 6.842021131645465,
 7.890087416548201]

In [30]:
df_user_info_influencers.sort_values(by="metrics",ascending=False,inplace=True)
df_user_info_influencers.index=[i for i in range(0,len(df_user_info_influencers))]
df_user_info_influencers.head()

,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes,is_followers,is_following,tweets_counts,retweets_counts,likes_counts,replies_counts,metrics
0,malditaternura,beto ortiz,Lima,,2011-03-06,261955501,46693,1920,3697619,13322,0,1,1,16,81,1,2.742715e+07
1,noticiAmerica,América Noticias,"Lima, Perú",americanoticias.pe,2009-08-13,65549291,240346,1278,2734216,3005,0,0,8,79,131,52,2.028256e+07
2,canalN_,Canal N,Peru,canaln.pe,2011-02-07,248692867,142597,526,2717299,10558,0,0,16,88,132,88,2.015753e+07
3,Capital967,Capital,"Lima, Peru",radio.capital.pe,2009-02-25,21889656,140935,812,2190585,3006,0,0,1,2,5,5,1.624837e+07
4,alvarezrodrich,Augusto Alvarez Rodrich,"Lima, Peru",,2010-02-09,112801997,27269,1308,2098239,2885,1,1,1,25,76,7,1.556410e+07


In [0]:
df_user_info_influencers.to_pickle("../data/intermediate/df_user_info_influencers.pkl")